# Scrape

### 1. Setup

The setup serves for:
- Importing libraries.
- Speficying utility functions.
- Setting up the chrome webdriver.
- Defining scraping constants.

In [33]:
# imports

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import ast
import numpy as np
import re

In [36]:
# set up ChromeDriver

options = Options()
# options.add_argument('--headless')  # save GUI rendering time
# options.add_argument('--disable-gpu')  # prevents errors in headless mode

service = Service('/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=service, options=options)

# open base page
BASE_URL = 'https://www.migros.ch/en'
driver.get(BASE_URL)
time.sleep(2)

In [35]:
# close driver
driver.quit()

In [14]:
# constants

# go to subcategory
SUBCAT_XPATH = '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/ng-component/main/div/ng-component/mo-category-level-1-children-categories/div/ul/li[{}]/div/div/a'
SUBCAT_XPATH = '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/ng-component/main/div/ng-component/mo-category-level-1-children-categories/div/ul/li[1]'
SUBCAT_XPATH = '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/ng-component/main/div/ng-component/mo-category-level-1-children-categories/div/ul/li[{}]/div/h3/a'
SUBCAT_CSS = '#main > div > ng-component > mo-category-level-1-children-categories > div > ul > li:nth-child({}) > div > h3 > a'
# go to page
TEMPLATE_URL = '?page={}'

# hand picked from base url
CATEGORIES = [
    'fruits-vegetables',
    'bread-pastries-breakfast',
    'pasta-condiments-canned-food',
    'snacks-sweets',
    'frozen-food',
    'drinks-coffee-tea',
    'wine-beer-spirits'
]

# I missed out on these, I'm stupid as f
CATEGORIES = [
    # 'meat-fish',
    'dairy-eggs-fresh-convenience-foo'
]

### 2. Scrape links

Considerations and steps:
- Since all products have pages have the same template,
it is convenient separating the link retrieval and the product specifics scraping.
- In this first step we navigate to category, sub-category, and scrape all products links from the subcategories subpages.
- All links are saved a dataframe, and then permanently saved to csv. 
- The same dataframe will be used to store the product specifics in the next step.

In [ ]:
# test for grids

# Open the target URL
url = 'https://www.migros.ch/en/category/meat-fish/meat-poultry?page=1'
driver.get(url)
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#main > div > ng-component > div > mo-items-display > div > ul > li'))
    )
except Exception as e:
    print(f"Error waiting for grid items to load: {e}")


grid_items = driver.find_elements(By.CSS_SELECTOR, '#main > div > ng-component > div > mo-items-display > div > ul > li')
num_items = len(grid_items)

print(f"Number of items found: {num_items}")

for i in range(1, num_items + 1):
    item_selector = f'#main > div > ng-component > div > mo-items-display > div > ul > li:nth-child({i})'
    try:
        item = driver.find_element(By.CSS_SELECTOR, item_selector)
        product_url = item.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
    except Exception as e:
        print(f"Could not retrieve item {i}: {e}")


In [ ]:
# list of dictionaries 
data = []

# big giant massive loop
for cat in CATEGORIES:
    url = BASE_URL + '/category/' + cat
    driver.get(url) 

    time.sleep(1)

    # dynamically find subcategories (li[n] elements)
    subcategory_urls = []
    li_index = 1

    while True:
        time.sleep(1)
        subcat_xpath = SUBCAT_XPATH.format(li_index)
        subcat_css = SUBCAT_CSS.format(li_index)
        try:
            # subcategory = driver.find_element(By.XPATH, SUBCAT_XPATH)
            subcategory = driver.find_element(By.CSS_SELECTOR, subcat_css)
            subcategory_url = subcategory.get_attribute('href')
            subcategory_urls.append(subcategory_url)

            print(f"Found subcategory link: {subcategory_url}")
            li_index += 1

        except NoSuchElementException:
            print("No more subcategory elements found.")
            break

    print(f"Subcategories found: {subcategory_urls}")

    for subcat_url in subcategory_urls:
        driver.get(subcat_url) 
        time.sleep(2)

        # extract products and go to the next page
        scrape_url = subcat_url + TEMPLATE_URL
        product_urls = []
        page = 1
        while True:
            time.sleep(1)
            url = scrape_url.format(page)
            driver.get(url)
            print(f"Scraping url: {url}")
            try:
                    
                try:
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '#main > div > ng-component > div > mo-items-display > div > ul > li'))
                    )
                except Exception as e:
                    print(f"Error waiting for grid items to load: {e}")


                grid_items = driver.find_elements(By.CSS_SELECTOR, '#main > div > ng-component > div > mo-items-display > div > ul > li')
                num_items = len(grid_items)

                if num_items <= 1:
                    print(f"No more products found on page {page}. Exiting.")
                    break

                print(f"Number of items found: {num_items}")

                for i in range(1, num_items + 1):
                    item_selector = f'#main > div > ng-component > div > mo-items-display > div > ul > li:nth-child({i})'
                    try:
                        item = driver.find_element(By.CSS_SELECTOR, item_selector)
                        product_url = item.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                        product_urls.append(product_url)
                    except Exception as e:
                        print(f"Could not retrieve item {i}: {e}")       
                print(f"Finished scraping page {page}.")
                page += 1
            
            except NoSuchElementException:
                print(f"Error on page {page}, stopping.")
                break
                
            # try:
            #     SEL = 'subcat'
            #     product_list = driver.find_elements(By.CLASS_NAME, SEL) 
            #     print("product_list", product_list)
            #     if not product_list:
            #         print(f"No more products found on page {page}. Exiting.")
            #         break
                
            #     for product in product_list:
            #         try:
            #             product_url = product.get_attribute('href')
            #             product_urls.append(product_url)
            #         except Exception as e:
            #             print(f"Error processing product: {e}")
            #             continue
                
            #     print(f"Finished scraping page {page}.")
            #     page += 1
            
            # except NoSuchElementException:
            #     print(f"Error on page {page}, stopping.")
            #     break

        # store
        current_data = {
            'category': cat,
            'sub_category': subcat_url.split('/')[-1],
            'url': product_urls
        }
        data.append(current_data)

        # report
        print(f"Finished scraping subcategory {subcat_url}")
        print(f"Data: {current_data}")

In [ ]:
len(data[1]['url'])

In [20]:
# convert to dataframe

df = pd.DataFrame(data)
df = df.explode('url')
df.to_csv('../data/product_categorization_and_urls_3.csv', index=False)

In [ ]:
df.head()
len(df)

In [23]:
# merge dataframes

df1 = pd.read_csv('../data/product_categorization_and_urls_2.csv')
df2 = pd.read_csv('../data/product_categorization_and_urls_3.csv')

df = pd.concat([df1, df2], ignore_index=True)
df.to_csv('../data/product_categorization_and_urls_1.csv', index=False)

### 3. Scrape product information

In [37]:
# adapt df 
df = pd.read_csv('../data/product_categorization_and_urls_1.csv')
len(df)

7243

In [40]:
# the xpaths seem to change each time I refresh the driver

In [43]:
# setup loop 

product_data = []

start_index = 6042
total = len(df)

# scrape product details given link 
for index, row in df.iloc[start_index:].iterrows():
    product_url = row['url']
    driver.get(product_url)
    time.sleep(0.5)  # Wait for the page to load

    try:
        # name, price, weight, price per unit
        try:
            name_selector = '#main > div > mo-product-detail-page > div.product-details-container.ng-star-inserted > div.product-image-info-container > div.product-core-container > div > mo-product-detail-core-information > div > div.core-product-title > mo-product-detail-title'
            product_name = driver.find_element(By.CSS_SELECTOR, name_selector).text
        except Exception as e:
            product_name = "N/A"
        try:
            product_price = driver.find_element(By.XPATH, '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[1]/div[2]/div/mo-product-detail-core-information/div/div[2]/mo-product-detail-price/span/span').text
        except Exception:
            product_price = "N/A"
        try:
            product_weight = driver.find_element(By.XPATH, '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[1]/div[2]/div/mo-product-detail-core-information/div/div[4]/mo-product-detail-quantity/span').text
        except Exception:
            product_weight = "N/A"
        try:
            price_per_unit = driver.find_element(By.XPATH, '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[1]/div[2]/div/mo-product-detail-core-information/div/div[4]/mo-product-detail-price-quantity/span/span').text
        except Exception:
            price_per_unit = "N/A"

        # nutritional values table
        nutrition_info = {}
        try:
            # why not using 
            table_body_selector = '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[2]/mo-product-detail-information/div/mo-expansion-panel/mat-expansion-panel[2]/div/div/mo-product-detail-nutrients-smartphone/m-slider/m-slider-container/div/div[2]/div[2]/ul/li[1]/table/tbody'
            table_body_selector = '#main > div > mo-product-detail-page > div.product-details-container.ng-star-inserted > div.product-information-container.ng-star-inserted > mo-product-detail-information > mo-tab-layout > div > div > div:nth-child(2) > mo-product-detail-nutrients-information > table > tbody'

            table_body = driver.find_element(By.CSS_SELECTOR, table_body_selector).get_attribute('outerHTML')
            soup = BeautifulSoup(table_body, 'html.parser')
            rows = soup.find_all('tr')

            for row in rows:
                nutrient_name = row.find('td', class_='mat-column-label').get_text(strip=True)
                nutrient_value = row.find('td', class_='mat-column-standardNutrient').get_text(strip=True)
                nutrition_info[nutrient_name] = nutrient_value
        except Exception as e:
            print(f"error table_content {index + 1}")

        # Store data in dict
        # Put dict in list
        current_data = {
            'url': product_url,
            'name': product_name,
            'weight': product_weight,
            'price': product_price,
            'price_per_unit': price_per_unit,
            'nutritional_values': nutrition_info
        }
        product_data.append(current_data)

        print(f"{index+1}/{total}")

    except Exception as e:
        print(f"Error processing product {index}: {str(e)}")
        continue

error table_content 6043
6043/7243
6044/7243
error table_content 6045
6045/7243
6046/7243
6047/7243
6048/7243
6049/7243
6050/7243
6051/7243
6052/7243
6053/7243
6054/7243
6055/7243
6056/7243
6057/7243
6058/7243
6059/7243
6060/7243
error table_content 6061
6061/7243
6062/7243
6063/7243
error table_content 6064
6064/7243
6065/7243
6066/7243
6067/7243
6068/7243
6069/7243
6070/7243
error table_content 6071
6071/7243
6072/7243
6073/7243
6074/7243
6075/7243
6076/7243
6077/7243
6078/7243
error table_content 6079
6079/7243
6080/7243
6081/7243
error table_content 6082
6082/7243
error table_content 6083
6083/7243
6084/7243
6085/7243
error table_content 6086
6086/7243
6087/7243
6088/7243
6089/7243
6090/7243
6091/7243
6092/7243
6093/7243
6094/7243
6095/7243
6096/7243
6097/7243
6098/7243
6099/7243
6100/7243
6101/7243
6102/7243
6103/7243
error table_content 6104
6104/7243
6105/7243
6106/7243
6107/7243
6108/7243
6109/7243
6110/7243
error table_content 6111
6111/7243
error table_content 6112
6112/7243


In [44]:
df = pd.DataFrame(product_data)
df.to_csv('../data/product_specifics_2.csv', index=False)

df.head()

,url,name,weight,price,price_per_unit,nutritional_values
0,https://www.migros.ch/en/product/205068000400,Joghurt Süssmost,150g,0.60,0.40/100g,{}
1,https://www.migros.ch/en/product/205069900400,Joghurt Kirsche,150g,0.80,0.53/100g,"{'Energy': '365 kJ (87 kcal)', 'Fatlow': '2.8 ..."
2,https://www.migros.ch/en/product/205070600600,Joghurt Birnen,150g,0.95,0.63/100g,{}
3,https://www.migros.ch/en/product/205080100000,M-Classic Joghurt Pfirsich-Passion,1kg,2.80,0.28/100g,"{'Energy': '351 kJ (84 kcal)', 'Fat': '2.9 g',..."
4,https://www.migros.ch/en/product/205228000000,Oh! Whey Protein Mango Joghurt,180g,1.60,0.89/100g,"{'Energy': '254 kJ (60 kcal)', 'Fatlow': '0.4 ..."


In [45]:
len(df)

1201

In [46]:
# close the driver
driver.quit()

### 4. Check datasets and merge

In [55]:
df1 = pd.read_csv('../data/product_categorization_and_urls_1.csv')
df2 = pd.read_csv('../data/product_specifics_1.csv')
df3 = pd.read_csv('../data/product_specifics_2.csv')

# something went wrong with three links
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df2.shape[0] + df3.shape[0])

(7243, 3)
(6042, 6)
(1201, 6)
7243


In [56]:
# check for duplicates
print(df1['url'].duplicated().sum())
print(df2['url'].duplicated().sum())
print(df3['url'].duplicated().sum())

# some urls are both in fruit and vegetables for example

69
52
10


In [58]:
# remove duplicates from df2, since the information is the same
df2 = df2.drop_duplicates(subset=['url'])
df3 = df3.drop_duplicates(subset=['url'])

print(df2['url'].duplicated().sum())
print(df3['url'].duplicated().sum())

len(df2) + len(df3)

0
0


7181

In [59]:
# merge the datasets
df2 = pd.concat([df2, df3], ignore_index=True)

In [60]:
print(df2['url'].duplicated().sum())

7


In [61]:
df2 = df2.drop_duplicates(subset=['url'])

In [62]:
# check scraping worked for all links 

missing_links = set(df1['url']) - set(df2['url']) 
len(missing_links)

0

In [64]:
# take care of nutritional_values column
df2['nutritional_values'][0]

"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g', 'of which saturateslow': '0.5 g', 'Carbohydrate': '0 g', 'of which sugarslow': '0 g', 'Fibre': '0 g', 'Protein': '25 g', 'Saltlow': '0.1 g'}"

In [65]:
# convert the strings to dictionaries
df2['nutritional_values'] = df2['nutritional_values'].apply(ast.literal_eval)

# separate columns
nutrition_df = pd.json_normalize(df2['nutritional_values'])

# merge new columns into df
df2 = df2.join(nutrition_df)

df2.head()

,url,name,weight,price,price_per_unit,nutritional_values,Energy,Fatlow,of which saturateslow,Carbohydrate,...,Chloride,Copper,Selen,Chrome,Molybdenum,Iodine,Zinc,of which starch,lactose,of which polyvalents alcohols
0,https://www.migros.ch/en/product/241102700300,Optigal · Chicken mini fillets,100 g,3.60,3.60/100g,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",499 kJ (118 kcal),2 g,0.5 g,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.migros.ch/en/product/241102800500,Optigal · Diced chicken,100 g,3.35,3.35/100g,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",499 kJ (118 kcal),2 g,0.5 g,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.migros.ch/en/product/241100500000,Optigal · Chicken breast medallion · Ideal for...,100 g,3.60,3.60/100g,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",499 kJ (118 kcal),2 g,0.5 g,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.migros.ch/en/product/227220026320,M-Classic · Beef Burger · classic,240g,6.40,2.67/100g,"{'Energy': '863 kJ (207 kcal)', 'Fatmedium': '...",863 kJ (207 kcal),NaN,NaN,3.3 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.migros.ch/en/product/220282000000,M-Classic · Beef entrecôte II,NaN,NaN,NaN,"{'Energy': '724 kJ (173 kcal)', 'Fatmedium': '...",724 kJ (173 kcal),NaN,NaN,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# replace {} with NaN in nutritional_values
df2['nutritional_values'] = df2['nutritional_values'].apply(lambda x: np.nan if x == {} else x)
df2.head()

,url,name,weight,price,price_per_unit,nutritional_values,Energy,Fatlow,of which saturateslow,Carbohydrate,...,Chloride,Copper,Selen,Chrome,Molybdenum,Iodine,Zinc,of which starch,lactose,of which polyvalents alcohols
0,https://www.migros.ch/en/product/241102700300,Optigal · Chicken mini fillets,100 g,3.60,3.60/100g,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",499 kJ (118 kcal),2 g,0.5 g,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.migros.ch/en/product/241102800500,Optigal · Diced chicken,100 g,3.35,3.35/100g,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",499 kJ (118 kcal),2 g,0.5 g,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.migros.ch/en/product/241100500000,Optigal · Chicken breast medallion · Ideal for...,100 g,3.60,3.60/100g,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",499 kJ (118 kcal),2 g,0.5 g,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.migros.ch/en/product/227220026320,M-Classic · Beef Burger · classic,240g,6.40,2.67/100g,"{'Energy': '863 kJ (207 kcal)', 'Fatmedium': '...",863 kJ (207 kcal),NaN,NaN,3.3 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.migros.ch/en/product/220282000000,M-Classic · Beef entrecôte II,NaN,NaN,NaN,"{'Energy': '724 kJ (173 kcal)', 'Fatmedium': '...",724 kJ (173 kcal),NaN,NaN,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# sanity check
print(len(df2.columns))
print(len(df2.columns.unique()))
print(len(df2))

61
61
7174


In [68]:
# merge the two dataframes
df = pd.merge(df1, df2, on='url', how='inner')
len(df)

7243

In [69]:
# duplicates come from the fact that some products are in multiple categories
# keep them 

In [70]:
df['weight'].unique()

array(['100 g', '240g', nan, '600g', '3 x 60g', '200g', '130g', '350g',
       '400g', '80g', '50g', '280g', '160g', '6 x 100g', '2 x 140g',
       'per kg', '230g', '140g', '300g', '250g', '115g', '150g', '360g',
       '3 x 222g', '2 x 250g', '260g', '2 x 120g', '800g', '180g', '500g',
       '550g', '5 x 130g', '640g', '4 x 100g', '100g', '1 piece', '220g',
       '2x140g', '6x30g', '450g', '2 x 350g', '2 x 60g', '120g',
       '5 x 18g', '70g', '2 x 100g', '290g', '2 x 160g', '40g', '330g',
       '8 x 20g', '90g', '190g', '153g', '75g', '320g', '4 x 50g', '225g',
       '2 x 150g', '125g', '110g', '45g', '170g', '95g', '85g', '4 x 22g',
       '60g', '118g', '105g', '2 x 190g', '97g', '540g', '3 x 120g',
       '270g', '2 x 110g', '2 x 400g', '5 x 100g', '2 x 90g', '310g',
       '490g', '2 x 180g', '2 x 175g', '3 x 160g', '2 x 260g', '560g',
       '1400g', '520g', '280 g', '440g', '129g', '30g', '340g', '2 x 80g',
       '390g', '58g', '3 x 95g', '2 x 95g', '25g', '4 x 120g', '4

In [79]:
# where 'per kg' put nan
df['weight'] = df['weight'].apply(lambda x: np.nan if isinstance(x, str) and 'per kg' in x else x)
df[df['weight'] == 'per kg']

,category,sub_category,url,name,weight,price,price_per_unit,nutritional_values,Energy,Fatlow,...,Chloride,Copper,Selen,Chrome,Molybdenum,Iodine,Zinc,of which starch,lactose,of which polyvalents alcohols


In [80]:
"""
Cases to cover:
'1300 Tabl.'
'nan'
'1 piece'
"Crème d'Or C"
'330ML'
'6x50cl'
'1dl'
'3 Stk.'
'3 Stück'
'3 x 90g'
'2x80g'
'6 x 12.5g'
'18 Balls'
'170G'
'1 Kilo'
"""
# Function to parse the weight column
def parse_weight(weight):
    if pd.isna(weight):
        return pd.Series([np.nan, np.nan, np.nan], index=['quantity', 'weight_unit', 'weight_per_unit'])
    
    weight = weight.strip().lower()
    
    # Match patterns like '<number> x <number><unit>'
    match = re.match(r'^(\d+)\s*x\s*(\d+(\.\d+)?)([a-z]+)$', weight)
    if match:
        quantity = int(match.group(1))
        weight_per_unit = float(match.group(2))
        weight_unit = match.group(4)
        if weight_unit in ['kg', 'kilo']:
            weight_unit = 'g'
            weight_per_unit *= 1000
        elif weight_unit == 'l':
            weight_unit = 'ml'
            weight_per_unit *= 1000
        elif weight_unit == 'dl':
            weight_unit = 'ml'
            weight_per_unit *= 100
        elif weight_unit == 'cl':
            weight_unit = 'ml'
            weight_per_unit *= 10
        return pd.Series([quantity, weight_unit, weight_per_unit], index=['quantity', 'weight_unit', 'weight_per_unit'])
    
    # Match patterns like '<number> pieces/stk/balls'
    match = re.match(r'^(\d+)\s*(stück|stk|pieces|balls)$', weight)
    if match:
        quantity = int(match.group(1))
        unit = match.group(2)
        if unit in ['stück', 'stk', 'pieces']:
            weight_unit = 'piece'
        elif unit == 'balls':
            weight_unit = 'balls'
        return pd.Series([quantity, weight_unit, np.nan], index=['quantity', 'weight_unit', 'weight_per_unit'])
    
    # Match patterns like '<number> <unit>' (e.g., '1 kg', '330ml')
    match = re.match(r'^(\d+(\.\d+)?)\s*([a-z]+)$', weight)
    if match:
        quantity = 1
        value = float(match.group(1))
        unit = match.group(3)
        if unit in ['kg', 'kilo']:
            weight_unit = 'g'
            weight_per_unit = value * 1000
        elif unit == 'g':
            weight_unit = 'g'
            weight_per_unit = value
        elif unit in ['ml', 'l']:
            weight_unit = 'ml'
            weight_per_unit = value * 1000 if unit == 'l' else value
        elif unit == 'cl':
            weight_unit = 'ml'
            weight_per_unit = value * 10
        elif unit == 'dl':
            weight_unit = 'ml'
            weight_per_unit = value * 100
        elif unit == 'piece':
            weight_unit = 'piece'
            weight_per_unit = np.nan
        elif unit == 'tabl':
            weight_unit = 'tabl'
            weight_per_unit = np.nan
        return pd.Series([quantity, weight_unit, weight_per_unit], index=['quantity', 'weight_unit', 'weight_per_unit'])
    
    # Default case for non-numeric information
    return pd.Series([np.nan, np.nan, np.nan], index=['quantity', 'weight_unit', 'weight_per_unit'])

# Apply the function to the DataFrame
df[['quantity', 'weight_unit', 'weight_per_unit']] = df['weight'].apply(parse_weight)

In [81]:
df.head()

,category,sub_category,url,name,weight,price,price_per_unit,nutritional_values,Energy,Fatlow,...,Chrome,Molybdenum,Iodine,Zinc,of which starch,lactose,of which polyvalents alcohols,quantity,weight_unit,weight_per_unit
0,meat-fish,meat-poultry,https://www.migros.ch/en/product/241102700300,Optigal · Chicken mini fillets,100 g,3.60,3.60/100g,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",499 kJ (118 kcal),2 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,100.0
1,meat-fish,meat-poultry,https://www.migros.ch/en/product/241102800500,Optigal · Diced chicken,100 g,3.35,3.35/100g,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",499 kJ (118 kcal),2 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,100.0
2,meat-fish,meat-poultry,https://www.migros.ch/en/product/241100500000,Optigal · Chicken breast medallion · Ideal for...,100 g,3.60,3.60/100g,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",499 kJ (118 kcal),2 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,100.0
3,meat-fish,meat-poultry,https://www.migros.ch/en/product/227220026320,M-Classic · Beef Burger · classic,240g,6.40,2.67/100g,"{'Energy': '863 kJ (207 kcal)', 'Fatmedium': '...",863 kJ (207 kcal),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,240.0
4,meat-fish,meat-poultry,https://www.migros.ch/en/product/220282000000,M-Classic · Beef entrecôte II,NaN,NaN,NaN,"{'Energy': '724 kJ (173 kcal)', 'Fatmedium': '...",724 kJ (173 kcal),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
df["quantity"].unique()

array([ 1., nan,  3.,  6.,  2.,  5.,  4.,  8., 10., 15., 20., 40., 12.,
        9.])

In [83]:
df["weight_unit"].unique()

array(['g', nan, 'piece', 'ml'], dtype=object)

In [84]:
df["weight_per_unit"].unique()

array([ 100. ,  240. ,    nan,  600. ,   60. ,  200. ,  130. ,  350. ,
        400. ,   80. ,   50. ,  280. ,  160. ,  140. ,  230. ,  300. ,
        250. ,  115. ,  150. ,  360. ,  222. ,  260. ,  120. ,  800. ,
        180. ,  500. ,  550. ,  640. ,  220. ,   30. ,  450. ,   18. ,
         70. ,  290. ,   40. ,  330. ,   20. ,   90. ,  190. ,  153. ,
         75. ,  320. ,  225. ,  125. ,  110. ,   45. ,  170. ,   95. ,
         85. ,   22. ,  118. ,  105. ,   97. ,  540. ,  270. ,  310. ,
        490. ,  175. ,  560. , 1400. ,  520. ,  440. ,  129. ,  340. ,
        390. ,   58. ,   25. ,  480. , 1000. ,  210. ,  193. ,  121. ,
        760. ,  750. ,  181. ,  195. ,  380. , 1500. ,  700. , 2000. ,
        720. ,  185. ,  370. ,  205. ,  215. ,  338. ,  235. ,    2.5,
        155. ,  365. ,  145. ,  305. ,  255. ,  226. ,  165. ,  138. ,
        460. ,  397. ,   12. ,  410. ,   15. ,  650. , 1750. ,  900. ,
        324. ,   35. ,  144. ,  192. ,  384. ,  128. ,  870. ,  580. ,
      

In [85]:
# remove the weight column
df = df.drop(columns=['weight'])

In [86]:
# parse price column to numeric
print(df['price'].unique())

# # remove dash '-' at end of string if present
df['price'] = df['price'].str.replace('–', '')
print(df['price'].unique())

# parse price column to numeric now that all values are numeric
df['price'] = df['price'].astype(float)
df['price'].unique()

['3.60' '3.35' '6.40' nan '3.95' '1.20' '1.40' '5.–' '1.80' '2.–' '4.20'
 '1.25' '3.20' '16.90' '5.85' '4.40' '4.60' '7.60' '5.65' '5.70' '4.75'
 '3.65' '1.15' '11.90' '2.90' '5.20' '3.70' '6.95' '7.90' '5.50' '5.90'
 '10.50' '9.90' '12.90' '20.95' '16.50' '4.30' '6.90' '5.95' '4.90' '7.95'
 '4.–' '12.95' '4.80' '4.05' '1.65' '1.30' '3.80' '1.70' '6.70' '9.95'
 '2.95' '5.45' '5.05' '8.90' '4.70' '3.50' '3.90' '8.45' '3.75' '9.50'
 '9.65' '8.60' '1.85' '5.10' '5.15' '2.80' '2.60' '4.50' '9.20' '5.40'
 '8.80' '10.80' '6.85' '4.85' '6.–' '6.35' '4.25' '6.20' '2.75' '4.15'
 '9.15' '5.30' '7.40' '1.50' '6.15' '2.15' '2.25' '2.40' '7.70' '5.25'
 '5.60' '9.–' '9.80' '3.25' '3.05' '7.–' '7.10' '7.80' '8.70' '11.–'
 '7.50' '2.35' '1.05' '6.30' '2.50' '10.40' '8.20' '6.05' '9.70' '8.50'
 '8.30' '3.–' '3.40' '8.40' '6.10' '28.50' '11.70' '3.45' '6.60' '16.–'
 '9.10' '8.75' '6.50' '9.60' '8.10' '2.05' '5.55' '1.55' '7.20' '4.10'
 '7.15' '7.45' '4.95' '1.90' '9.30' '6.80' '1.95' '3.10' '7.25' '6.55

array([3.600e+00, 3.350e+00, 6.400e+00,       nan, 3.950e+00, 1.200e+00,
       1.400e+00, 5.000e+00, 1.800e+00, 2.000e+00, 4.200e+00, 1.250e+00,
       3.200e+00, 1.690e+01, 5.850e+00, 4.400e+00, 4.600e+00, 7.600e+00,
       5.650e+00, 5.700e+00, 4.750e+00, 3.650e+00, 1.150e+00, 1.190e+01,
       2.900e+00, 5.200e+00, 3.700e+00, 6.950e+00, 7.900e+00, 5.500e+00,
       5.900e+00, 1.050e+01, 9.900e+00, 1.290e+01, 2.095e+01, 1.650e+01,
       4.300e+00, 6.900e+00, 5.950e+00, 4.900e+00, 7.950e+00, 4.000e+00,
       1.295e+01, 4.800e+00, 4.050e+00, 1.650e+00, 1.300e+00, 3.800e+00,
       1.700e+00, 6.700e+00, 9.950e+00, 2.950e+00, 5.450e+00, 5.050e+00,
       8.900e+00, 4.700e+00, 3.500e+00, 3.900e+00, 8.450e+00, 3.750e+00,
       9.500e+00, 9.650e+00, 8.600e+00, 1.850e+00, 5.100e+00, 5.150e+00,
       2.800e+00, 2.600e+00, 4.500e+00, 9.200e+00, 5.400e+00, 8.800e+00,
       1.080e+01, 6.850e+00, 4.850e+00, 6.000e+00, 6.350e+00, 4.250e+00,
       6.200e+00, 2.750e+00, 4.150e+00, 9.150e+00, 

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7243 entries, 0 to 7242
Data columns (total 65 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   category                                    7243 non-null   object 
 1   sub_category                                7243 non-null   object 
 2   url                                         7243 non-null   object 
 3   name                                        6961 non-null   object 
 4   price                                       5149 non-null   float64
 5   price_per_unit                              5145 non-null   object 
 6   nutritional_values                          5197 non-null   object 
 7   Energy                                      5186 non-null   object 
 8   Fatlow                                      616 non-null    object 
 9   of which saturateslow                       950 non-null    object 
 10  Carbohydrate

In [88]:
# print all of them to see what we are working with
for up in df['price_per_unit'].unique():
    print(up)

# split on "/"
df[['price_per_unit', 'price_unit']] = df['price_per_unit'].str.split('/', expand=True)
print(df['price_unit'].unique())
print(df['price_per_unit'].unique())

3.60/100g
3.35/100g
2.67/100g
nan
3.95/100g
1.20/100g
1.40/100g
5.–/100g
1.80/100g
2.–/100g
4.20/100g
1.25/100g
3.20/100g
2.82/100g
5.85/100g
4.40/100g
4.60/100g
7.60/100g
5.65/100g
5.70/100g
2.64/100g
3.65/100g
1.15/100g
11.90/100g
2.90/100g
2.60/100g
3.69/100g
5.35/100g
7.90/100g
5.50/100g
5.90/100g
10.50/100g
2.83/100g
5.99/100g
4.13/100g
4.30/100g
6.90/100g
7.44/100g
9.80/100g
2.84/100g
1.43/100g
8.09/100g
4.80/100g
4.05/100g
1.65/100g
1.30/100g
3.80/100g
1.70/100g
6.70/100g
1.66/100g
2.95/100g
1.36/100g
5.05/100g
8.90/100g
4.70/100g
5.20/100g
3.50/100g
3.90/100g
8.45/100g
3.75/100g
9.50/100g
9.65/100g
8.60/100g
1.85/100g
6.95/100g
5.10/100g
5.15/100g
2.80/100g
4.50/100g
9.20/100g
5.40/100g
8.80/100g
10.80/100g
6.85/100g
4.90/100g
4.85/100g
6.–/100g
6.35/100g
4.75/100g
4.25/100g
6.20/100g
2.75/100g
4.15/100g
9.15/100g
5.30/100g
7.40/100g
1.50/100g
6.15/100g
6.40/100g
2.15/100g
2.25/100g
2.40/100g
7.70/100g
5.25/100g
5.60/100g
9.–/100g
3.25/100g
3.05/100g
5.95/100g
7.–/100g
7.10/100

In [89]:
df['price_per_unit'].unique()

array(['3.60', '3.35', '2.67', nan, '3.95', '1.20', '1.40', '5.–', '1.80',
       '2.–', '4.20', '1.25', '3.20', '2.82', '5.85', '4.40', '4.60',
       '7.60', '5.65', '5.70', '2.64', '3.65', '1.15', '11.90', '2.90',
       '2.60', '3.69', '5.35', '7.90', '5.50', '5.90', '10.50', '2.83',
       '5.99', '4.13', '4.30', '6.90', '7.44', '9.80', '2.84', '1.43',
       '8.09', '4.80', '4.05', '1.65', '1.30', '3.80', '1.70', '6.70',
       '1.66', '2.95', '1.36', '5.05', '8.90', '4.70', '5.20', '3.50',
       '3.90', '8.45', '3.75', '9.50', '9.65', '8.60', '1.85', '6.95',
       '5.10', '5.15', '2.80', '4.50', '9.20', '5.40', '8.80', '10.80',
       '6.85', '4.90', '4.85', '6.–', '6.35', '4.75', '4.25', '6.20',
       '2.75', '4.15', '9.15', '5.30', '7.40', '1.50', '6.15', '6.40',
       '2.15', '2.25', '2.40', '7.70', '5.25', '5.60', '9.–', '3.25',
       '3.05', '5.95', '7.–', '7.10', '7.80', '8.70', '11.–', '7.50',
       '2.35', '1.05', '4.–', '6.30', '2.50', '10.40', '8.20', '6.05',
   

In [90]:
# remove dash '-' at end of string if present
df['price_per_unit'] = df['price_per_unit'].str.replace('–', '0')
print(df['price_per_unit'].unique())

['3.60' '3.35' '2.67' nan '3.95' '1.20' '1.40' '5.0' '1.80' '2.0' '4.20'
 '1.25' '3.20' '2.82' '5.85' '4.40' '4.60' '7.60' '5.65' '5.70' '2.64'
 '3.65' '1.15' '11.90' '2.90' '2.60' '3.69' '5.35' '7.90' '5.50' '5.90'
 '10.50' '2.83' '5.99' '4.13' '4.30' '6.90' '7.44' '9.80' '2.84' '1.43'
 '8.09' '4.80' '4.05' '1.65' '1.30' '3.80' '1.70' '6.70' '1.66' '2.95'
 '1.36' '5.05' '8.90' '4.70' '5.20' '3.50' '3.90' '8.45' '3.75' '9.50'
 '9.65' '8.60' '1.85' '6.95' '5.10' '5.15' '2.80' '4.50' '9.20' '5.40'
 '8.80' '10.80' '6.85' '4.90' '4.85' '6.0' '6.35' '4.75' '4.25' '6.20'
 '2.75' '4.15' '9.15' '5.30' '7.40' '1.50' '6.15' '6.40' '2.15' '2.25'
 '2.40' '7.70' '5.25' '5.60' '9.0' '3.25' '3.05' '5.95' '7.0' '7.10'
 '7.80' '8.70' '11.0' '7.50' '2.35' '1.05' '4.0' '6.30' '2.50' '10.40'
 '8.20' '6.05' '9.70' '0.85' '8.30' '3.0' '3.40' '8.40' '0.61' '28.50'
 '11.70' '3.45' '6.60' '16.0' '9.10' '8.75' '6.50' '9.60' '8.10' '2.05'
 '0.25' '5.55' '1.55' '7.20' '4.10' '7.15' '7.45' '4.95' '1.90' '0.84'
 '9

In [91]:
# parse to float
df['price_per_unit'] = df['price_per_unit'].astype(float)
print(df['price_per_unit'].unique())

# sort by price_per_unit and print first 5
df.sort_values(by='price_per_unit', ascending=False).head(20)

[3.600e+00 3.350e+00 2.670e+00       nan 3.950e+00 1.200e+00 1.400e+00
 5.000e+00 1.800e+00 2.000e+00 4.200e+00 1.250e+00 3.200e+00 2.820e+00
 5.850e+00 4.400e+00 4.600e+00 7.600e+00 5.650e+00 5.700e+00 2.640e+00
 3.650e+00 1.150e+00 1.190e+01 2.900e+00 2.600e+00 3.690e+00 5.350e+00
 7.900e+00 5.500e+00 5.900e+00 1.050e+01 2.830e+00 5.990e+00 4.130e+00
 4.300e+00 6.900e+00 7.440e+00 9.800e+00 2.840e+00 1.430e+00 8.090e+00
 4.800e+00 4.050e+00 1.650e+00 1.300e+00 3.800e+00 1.700e+00 6.700e+00
 1.660e+00 2.950e+00 1.360e+00 5.050e+00 8.900e+00 4.700e+00 5.200e+00
 3.500e+00 3.900e+00 8.450e+00 3.750e+00 9.500e+00 9.650e+00 8.600e+00
 1.850e+00 6.950e+00 5.100e+00 5.150e+00 2.800e+00 4.500e+00 9.200e+00
 5.400e+00 8.800e+00 1.080e+01 6.850e+00 4.900e+00 4.850e+00 6.000e+00
 6.350e+00 4.750e+00 4.250e+00 6.200e+00 2.750e+00 4.150e+00 9.150e+00
 5.300e+00 7.400e+00 1.500e+00 6.150e+00 6.400e+00 2.150e+00 2.250e+00
 2.400e+00 7.700e+00 5.250e+00 5.600e+00 9.000e+00 3.250e+00 3.050e+00
 5.950

,category,sub_category,url,name,price,price_per_unit,nutritional_values,Energy,Fatlow,of which saturateslow,...,Molybdenum,Iodine,Zinc,of which starch,lactose,of which polyvalents alcohols,quantity,weight_unit,weight_per_unit,price_unit
7012,dairy-eggs-fresh-convenience-foo,fresh-convenience-food,https://www.migros.ch/en/product/131356900200,Migros Daily Bulgur Karotten Salatschale,5.20,2600.00,"{'Energy': '491 kJ (117 kcal)', 'Fat': '3.4 g'...",733 kJ (176 kcal),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,200.0,100g
3148,meat-fish,fish,https://www.migros.ch/en/product/251513400000,Sélection · Caviar · of the Swiss Alps,49.50,247.50,"{'Energy': '989 kJ (237 kcal)', 'Fatmedium': '...",989 kJ (237 kcal),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,20.0,100g
3113,meat-fish,fish,https://www.migros.ch/en/product/251513300000,Sélection Königskaviar,49.00,245.00,"{'Energy': '1070 kJ (256 kcal)', 'Fatmedium': ...",1070 kJ (256 kcal),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,20.0,100g
552,meat-fish,meat-poultry,https://www.migros.ch/en/product/220262000192,Wagyu Filet Mittelstück,28.50,28.50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,100.0,100g
3011,meat-fish,fish,https://www.migros.ch/en/product/250409500100,ASC Lachs Gemüsepfanne,5.75,23.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,100.0,100g
3114,meat-fish,fish,https://www.migros.ch/en/product/251514240000,Gorbuscha Lachsrogen,11.50,23.00,"{'Energy': '1037 kJ (248 kcal)', 'Fatmedium': ...",1037 kJ (248 kcal),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,50.0,100g
1619,meat-fish,cold-cuts,https://www.migros.ch/en/product/243963406000,Delpeyrat · Whole duck foie gras · Torchon,39.50,21.94,"{'Energy': '2268 kJ (550 kcal)', 'Fathigh': '5...",2268 kJ (550 kcal),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,180.0,100g
2928,meat-fish,fish,https://www.migros.ch/en/product/131558000000,Sushi Yokohama,21.90,21.90,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,100.0,100g
1823,meat-fish,cold-cuts,https://www.migros.ch/en/product/233014082700,Sélection · Raw ham Pata Negra,12.95,21.58,"{'Energy': '1594 kJ (383 kcal)', 'Fathigh': '2...",1594 kJ (383 kcal),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,60.0,100g
1618,meat-fish,cold-cuts,https://www.migros.ch/en/product/243964550000,Labeyrie Le Classique · Block of duck foie gra...,21.00,21.00,"{'Energy': '2104 kJ (511 kcal)', 'Fathigh': '5...",2104 kJ (511 kcal),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,g,100.0,100g


In [92]:
# the anomalies are already present from the scraping, although the website displays the price per unit correctly

In [93]:
# Energy column

# Function to split the Energy column into energy_kJ and energy_kcal
def split_energy(energy):
    if pd.isna(energy) or energy.strip() == '':
        return np.nan, np.nan
    try:
        # Extract the kJ value (before the first space)
        energy_kJ = energy.split(' ')[0]
        # Extract the kcal value (inside the parentheses)
        energy_kcal = energy.split('(')[1].replace('kcal)', '').strip()
        return float(energy_kJ), float(energy_kcal)
    except (IndexError, ValueError):
        return np.nan, np.nan

# Apply the function to split the Energy column
df[['energy_kJ', 'energy_kcal']] = df['Energy'].apply(lambda x: pd.Series(split_energy(x)))

# Drop the original Energy column
df.drop(columns=['Energy'], inplace=True)

# Display the result
df.head()


,category,sub_category,url,name,price,price_per_unit,nutritional_values,Fatlow,of which saturateslow,Carbohydrate,...,Zinc,of which starch,lactose,of which polyvalents alcohols,quantity,weight_unit,weight_per_unit,price_unit,energy_kJ,energy_kcal
0,meat-fish,meat-poultry,https://www.migros.ch/en/product/241102700300,Optigal · Chicken mini fillets,3.60,3.60,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",2 g,0.5 g,0 g,...,NaN,NaN,NaN,NaN,1.0,g,100.0,100g,499.0,118.0
1,meat-fish,meat-poultry,https://www.migros.ch/en/product/241102800500,Optigal · Diced chicken,3.35,3.35,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",2 g,0.5 g,0 g,...,NaN,NaN,NaN,NaN,1.0,g,100.0,100g,499.0,118.0
2,meat-fish,meat-poultry,https://www.migros.ch/en/product/241100500000,Optigal · Chicken breast medallion · Ideal for...,3.60,3.60,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",2 g,0.5 g,0 g,...,NaN,NaN,NaN,NaN,1.0,g,100.0,100g,499.0,118.0
3,meat-fish,meat-poultry,https://www.migros.ch/en/product/227220026320,M-Classic · Beef Burger · classic,6.40,2.67,"{'Energy': '863 kJ (207 kcal)', 'Fatmedium': '...",NaN,NaN,3.3 g,...,NaN,NaN,NaN,NaN,1.0,g,240.0,100g,863.0,207.0
4,meat-fish,meat-poultry,https://www.migros.ch/en/product/220282000000,M-Classic · Beef entrecôte II,NaN,NaN,"{'Energy': '724 kJ (173 kcal)', 'Fatmedium': '...",NaN,NaN,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,724.0,173.0


In [95]:
df.columns

Index(['category', 'sub_category', 'url', 'name', 'price', 'price_per_unit',
       'nutritional_values', 'Fatlow', 'of which saturateslow', 'Carbohydrate',
       'of which sugarslow', 'Fibre', 'Protein', 'Saltlow', 'Fatmedium',
       'of which saturateshigh', 'Salthigh', 'of which saturatesmedium',
       'Saltmedium', 'Fat', 'of which saturates', 'of which sugars', 'Salt',
       'Fathigh', 'Energy value in kcal', 'Sodium', 'of which sugarsmedium',
       'of which sugarshigh', 'Energy value in kJ', 'Vitamin B12', 'Iron',
       'Vitamin B2 (riboflavin)', 'of which mono-unsaturates', 'Calcium',
       'Vitamin A', 'Vitamin E', 'Vitamin D', 'vegetable sterine',
       'Vitamin B1 (thiamin)', 'Niacin', 'Vitamin B6', 'Biotin',
       'Pantothenic acid', 'folic acid', 'folic acid / folate',
       'Alpha linolenic acid (omega-3 fatty acids)', 'Phosphorus', 'Magnesium',
       'Vitamin K', 'Vitamin C', 'Potassium', 'Chloride', 'Copper', 'Selen',
       'Chrome', 'Molybdenum', 'Iodine', 

In [96]:
cols_to_parse = ['Fatlow', 'of which saturateslow', 'Carbohydrate',
       'of which sugarslow', 'Fibre', 'Protein', 'Saltlow', 'Fatmedium',
       'of which saturateshigh', 'Salthigh', 'of which saturatesmedium',
       'Saltmedium', 'Fat', 'of which saturates', 'of which sugars', 'Salt',
       'Fathigh', 'Energy value in kcal', 'Sodium', 'of which sugarsmedium',
       'of which sugarshigh', 'Energy value in kJ', 'Vitamin B12', 'Iron',
       'Vitamin B2 (riboflavin)', 'of which mono-unsaturates', 'Calcium',
       'Vitamin A', 'Vitamin E', 'Vitamin D', 'vegetable sterine',
       'Vitamin B1 (thiamin)', 'Niacin', 'Vitamin B6', 'Biotin',
       'Pantothenic acid', 'folic acid', 'folic acid / folate',
       'Alpha linolenic acid (omega-3 fatty acids)', 'Phosphorus', 'Magnesium',
       'Vitamin K', 'Vitamin C', 'Potassium', 'Chloride', 'Copper', 'Selen',
       'Chrome', 'Molybdenum', 'Iodine', 'Zinc', 'of which starch', 'lactose',
       'of which polyvalents alcohols']

In [97]:
# Function to parse the weight values
def parse_value(value):
    if pd.isna(value) or value.strip() == '':
        return np.nan
    try:
        # Remove leading '~', '<' and other characters that aren't part of the numeric value
        value = value.replace('~', '').replace('<', '').strip()
        # Extract the numeric part
        numeric_value = ''.join([char for char in value if char.isdigit() or char == '.' or char == ','])
        # Convert to float (handle European comma notation)
        return float(numeric_value.replace(',', '.'))
    except (ValueError, TypeError):
        return np.nan

# Apply the function to parse the numeric part of the values
for col in cols_to_parse:
    df[col] = df[col].apply(parse_value)
    df.rename(columns={col: col + ' (g)'}, inplace=True)

# Display the result
df.head()

,category,sub_category,url,name,price,price_per_unit,nutritional_values,Fatlow (g),of which saturateslow (g),Carbohydrate (g),...,Zinc (g),of which starch (g),lactose (g),of which polyvalents alcohols (g),quantity,weight_unit,weight_per_unit,price_unit,energy_kJ,energy_kcal
0,meat-fish,meat-poultry,https://www.migros.ch/en/product/241102700300,Optigal · Chicken mini fillets,3.60,3.60,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",2.0,0.5,0.0,...,NaN,NaN,NaN,NaN,1.0,g,100.0,100g,499.0,118.0
1,meat-fish,meat-poultry,https://www.migros.ch/en/product/241102800500,Optigal · Diced chicken,3.35,3.35,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",2.0,0.5,0.0,...,NaN,NaN,NaN,NaN,1.0,g,100.0,100g,499.0,118.0
2,meat-fish,meat-poultry,https://www.migros.ch/en/product/241100500000,Optigal · Chicken breast medallion · Ideal for...,3.60,3.60,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",2.0,0.5,0.0,...,NaN,NaN,NaN,NaN,1.0,g,100.0,100g,499.0,118.0
3,meat-fish,meat-poultry,https://www.migros.ch/en/product/227220026320,M-Classic · Beef Burger · classic,6.40,2.67,"{'Energy': '863 kJ (207 kcal)', 'Fatmedium': '...",NaN,NaN,3.3,...,NaN,NaN,NaN,NaN,1.0,g,240.0,100g,863.0,207.0
4,meat-fish,meat-poultry,https://www.migros.ch/en/product/220282000000,M-Classic · Beef entrecôte II,NaN,NaN,"{'Energy': '724 kJ (173 kcal)', 'Fatmedium': '...",NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,724.0,173.0


In [98]:
# convert cols to strings
df['category'] = df['category'].astype(str)
df['sub_category'] = df['sub_category'].astype(str)
df['url'] = df['url'].astype(str)
df['name'] = df['name'].astype(str)
df['price_unit'] = df['price_unit'].astype(str)
df['weight_unit'] = df['weight_unit'].astype(str)

In [99]:

# Display the result
df.head()

,category,sub_category,url,name,price,price_per_unit,nutritional_values,Fatlow (g),of which saturateslow (g),Carbohydrate (g),...,Zinc (g),of which starch (g),lactose (g),of which polyvalents alcohols (g),quantity,weight_unit,weight_per_unit,price_unit,energy_kJ,energy_kcal
0,meat-fish,meat-poultry,https://www.migros.ch/en/product/241102700300,Optigal · Chicken mini fillets,3.60,3.60,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",2.0,0.5,0.0,...,NaN,NaN,NaN,NaN,1.0,g,100.0,100g,499.0,118.0
1,meat-fish,meat-poultry,https://www.migros.ch/en/product/241102800500,Optigal · Diced chicken,3.35,3.35,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",2.0,0.5,0.0,...,NaN,NaN,NaN,NaN,1.0,g,100.0,100g,499.0,118.0
2,meat-fish,meat-poultry,https://www.migros.ch/en/product/241100500000,Optigal · Chicken breast medallion · Ideal for...,3.60,3.60,"{'Energy': '499 kJ (118 kcal)', 'Fatlow': '2 g...",2.0,0.5,0.0,...,NaN,NaN,NaN,NaN,1.0,g,100.0,100g,499.0,118.0
3,meat-fish,meat-poultry,https://www.migros.ch/en/product/227220026320,M-Classic · Beef Burger · classic,6.40,2.67,"{'Energy': '863 kJ (207 kcal)', 'Fatmedium': '...",NaN,NaN,3.3,...,NaN,NaN,NaN,NaN,1.0,g,240.0,100g,863.0,207.0
4,meat-fish,meat-poultry,https://www.migros.ch/en/product/220282000000,M-Classic · Beef entrecôte II,NaN,NaN,"{'Energy': '724 kJ (173 kcal)', 'Fatmedium': '...",NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,724.0,173.0


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7243 entries, 0 to 7242
Data columns (total 67 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   category                                        7243 non-null   object 
 1   sub_category                                    7243 non-null   object 
 2   url                                             7243 non-null   object 
 3   name                                            7243 non-null   object 
 4   price                                           5149 non-null   float64
 5   price_per_unit                                  5145 non-null   float64
 6   nutritional_values                              5197 non-null   object 
 7   Fatlow (g)                                      614 non-null    float64
 8   of which saturateslow (g)                       948 non-null    float64
 9   Carbohydrate (g)                         

In [104]:
# reorder columns: 
"""
0   category                                        7243 non-null   object 
 1   sub_category                                    7243 non-null   object 
 2   url                                             7243 non-null   object 
 3   name                                            7243 non-null   object 
 4   price                                           5149 non-null   float64
 5   price_per_unit                                  5145 non-null   float64
 6   nutritional_values                              5197 non-null   object 
 7   Fatlow (g)                                      614 non-null    float64
 8   of which saturateslow (g)                       948 non-null    float64
 9   Carbohydrate (g)                                5171 non-null   float64
 10  of which sugarslow (g)                          3730 non-null   float64
 11  Fibre (g)                                       4574 non-null   float64
 12  Protein (g)                                     5180 non-null   float64
 13  Saltlow (g)                                     1085 non-null   float64
 14  Fatmedium (g)                                   1520 non-null   float64
 15  of which saturateshigh (g)                      2257 non-null   float64
 16  Salthigh (g)                                    1853 non-null   float64
 17  of which saturatesmedium (g)                    1044 non-null   float64
 18  Saltmedium (g)                                  1316 non-null   float64
 19  Fat (g)                                         928 non-null    float64
 20  of which saturates (g)                          712 non-null    float64
 21  of which sugars (g)                             710 non-null    float64
 22  Salt (g)                                        924 non-null    float64
 23  Fathigh (g)                                     2119 non-null   float64
 24  Energy value in kcal (g)                        9 non-null      float64
 25  Sodium (g)                                      42 non-null     float64
 26  of which sugarsmedium (g)                       476 non-null    float64
 27  of which sugarshigh (g)                         42 non-null     float64
 28  Energy value in kJ (g)                          1 non-null      float64
 29  Vitamin B12 (g)                                 39 non-null     float64
 30  Iron (g)                                        17 non-null     float64
 31  Vitamin B2 (riboflavin) (g)                     37 non-null     float64
 32  of which mono-unsaturates (g)                   21 non-null     float64
 33  Calcium (g)                                     121 non-null    float64
 34  Vitamin A (g)                                   12 non-null     float64
 35  Vitamin E (g)                                   27 non-null     float64
 36  Vitamin D (g)                                   55 non-null     float64
 37  vegetable sterine (g)                           1 non-null      float64
 38  Vitamin B1 (thiamin) (g)                        27 non-null     float64
 39  Niacin (g)                                      11 non-null     float64
 40  Vitamin B6 (g)                                  30 non-null     float64
 41  Biotin (g)                                      11 non-null     float64
 42  Pantothenic acid (g)                            10 non-null     float64
 43  folic acid (g)                                  8 non-null      float64
 44  folic acid / folate (g)                         9 non-null      float64
 45  Alpha linolenic acid (omega-3 fatty acids) (g)  1 non-null      float64
 46  Phosphorus (g)                                  12 non-null     float64
 47  Magnesium (g)                                   11 non-null     float64
 48  Vitamin K (g)                                   7 non-null      float64
 49  Vitamin C (g)                                   10 non-null     float64
 50  Potassium (g)                                   10 non-null     float64
 51  Chloride (g)                                    7 non-null      float64
 52  Copper (g)                                      7 non-null      float64
 53  Selen (g)                                       7 non-null      float64
 54  Chrome (g)                                      7 non-null      float64
 55  Molybdenum (g)                                  7 non-null      float64
 56  Iodine (g)                                      9 non-null      float64
 57  Zinc (g)                                        2 non-null      float64
 58  of which starch (g)                             5 non-null      float64
 59  lactose (g)                                     1 non-null      float64
 60  of which polyvalents alcohols (g)               1 non-null      float64
 61  quantity                                        5135 non-null   float64
 62  weight_unit                                     7243 non-null   object 
 63  weight_per_unit                                 5027 non-null   float64
 64  price_unit                                      7243 non-null   object 
 65  energy_kJ                                       4652 non-null   float64
 66  energy_kcal                                     4652 non-null   float64
"""

df = df[['name', 'url', 'category', 'sub_category', 'price', 'price_per_unit', 'price_unit', 'quantity', 'weight_per_unit', 'weight_unit', 'energy_kJ', 'energy_kcal', 'Fatlow (g)', 'of which saturateslow (g)', 'Carbohydrate (g)', 'of which sugarslow (g)', 'Fibre (g)', 'Protein (g)', 'Saltlow (g)', 'Fatmedium (g)', 'of which saturateshigh (g)', 'Salthigh (g)', 'of which saturatesmedium (g)', 'Saltmedium (g)', 'Fat (g)', 'of which saturates (g)', 'of which sugars (g)', 'Salt (g)', 'Fathigh (g)', 'Energy value in kcal (g)', 'Sodium (g)', 'of which sugarsmedium (g)', 'of which sugarshigh (g)', 'Energy value in kJ (g)', 'Vitamin B12 (g)', 'Iron (g)', 'Vitamin B2 (riboflavin) (g)', 'of which mono-unsaturates (g)', 'Calcium (g)', 'Vitamin A (g)', 'Vitamin E (g)', 'Vitamin D (g)', 'vegetable sterine (g)', 'Vitamin B1 (thiamin) (g)', 'Niacin (g)', 'Vitamin B6 (g)', 'Biotin (g)', 'Pantothenic acid (g)', 'folic acid (g)', 'folic acid / folate (g)', 'Alpha linolenic acid (omega-3 fatty acids) (g)', 'Phosphorus (g)', 'Magnesium (g)', 'Vitamin K (g)', 'Vitamin C (g)', 'Potassium (g)', 'Chloride (g)', 'Copper (g)', 'Selen (g)', 'Chrome (g)', 'Molybdenum (g)', 'Iodine (g)', 'Zinc (g)', 'of which starch (g)', 'lactose (g)', 'of which polyvalents alcohols (g)']]

In [105]:
# rename all columns to lowercase
df.columns = df.columns.str.lower()
df.columns

Index(['name', 'url', 'category', 'sub_category', 'price', 'price_per_unit',
       'price_unit', 'quantity', 'weight_per_unit', 'weight_unit', 'energy_kj',
       'energy_kcal', 'fatlow (g)', 'of which saturateslow (g)',
       'carbohydrate (g)', 'of which sugarslow (g)', 'fibre (g)',
       'protein (g)', 'saltlow (g)', 'fatmedium (g)',
       'of which saturateshigh (g)', 'salthigh (g)',
       'of which saturatesmedium (g)', 'saltmedium (g)', 'fat (g)',
       'of which saturates (g)', 'of which sugars (g)', 'salt (g)',
       'fathigh (g)', 'energy value in kcal (g)', 'sodium (g)',
       'of which sugarsmedium (g)', 'of which sugarshigh (g)',
       'energy value in kj (g)', 'vitamin b12 (g)', 'iron (g)',
       'vitamin b2 (riboflavin) (g)', 'of which mono-unsaturates (g)',
       'calcium (g)', 'vitamin a (g)', 'vitamin e (g)', 'vitamin d (g)',
       'vegetable sterine (g)', 'vitamin b1 (thiamin) (g)', 'niacin (g)',
       'vitamin b6 (g)', 'biotin (g)', 'pantothenic acid (g)

In [106]:
# save final dataframe
df.to_csv('../data/final_dataset_2.csv', index=False)

In [107]:
# final check
len(df)

7243

In [108]:
# read final_dataset 1
df_1 = pd.read_csv('../data/final_dataset_.csv')
df_2 = pd.read_csv('../data/final_dataset_2.csv')

# merge the two dataframes
df = pd.concat([df_1, df_2], ignore_index=True)

In [109]:
len(df.columns)

79

In [110]:
df.head()

,name,url,category,sub_category,price,price_per_unit,price_unit,quantity,weight_per_unit,weight_unit,...,bicarbonate (or hydrogen carbonate) (g),nitrate (g),sulfate (g),chloride (g),sulphate 4+ (g),minerals (g),vegetable sterine (g),chrome (g),molybdenum (g),lactose (g)
0,Migros Bio · blueberries,https://www.migros.ch/en/product/265680802400,fruits-vegetables,fruits,7.90,3.16,100g,1.0,250.0,g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dates,https://www.migros.ch/en/product/264601302100,fruits-vegetables,fruits,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Avocado,https://www.migros.ch/en/product/264500313200,fruits-vegetables,fruits,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bio Fairtrade · Bananas,https://www.migros.ch/en/product/264280101000,fruits-vegetables,fruits,1.95,NaN,NaN,1.0,1000.0,g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Migros Bio · Avocado,https://www.migros.ch/en/product/264580113200,fruits-vegetables,fruits,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
len(df)

16845

In [112]:
df.to_csv('../data/final_dataset.csv', index=False)

In [2]:
# merge product_and_categorization datasets
import pandas as pd
df_1 = pd.read_csv('../data/product_categorization_and_urls_1.csv')
df_2 = pd.read_csv('../data/product_categorization_and_urls_.csv')

df = pd.concat([df_1, df_2], ignore_index=True)

df.to_csv('../data/product_categorization_and_urls.csv', index=False)

In [3]:
len(df)

16845